### Getting started:

##### Connection Setup:
   
   Connect two Supernovas between them via their I3C High Voltage connectors, and connect also each one to the computer via the USB port. One will act as an I3C controller and the other one as an I3C target.

   The target Supernova acts as a 2 bytes-addressable and non-linear memory of 1024 bytes. The memory can be accessed via USB when using the `i3cTargetWriteMemory` or `i3cTargetReadMemory` methods. Or it can be accessed through the I3C bus when the user requests an I3C write, I3C read or I3C write-read to the Supernova acting as the I3C controller addressing the Supernova acting as a target.

##### Expected Messages and Notifications:

   Both Supernovas will return a response to their requests. However, when a request is sent to the Supernova acting as a the controller to issue an I3C transfer, whether it is a private transfer or a CCC, the Supernova controller will send a response to the USB Host (in this case, this notebook), but the Supernova acting as a target will send also an asynchronous notification to notify the I3C bus event that occurred.
   
##### Testing Process:
   - Run each cell in order.
   - Look for immediate feedback:
     - The response or the notification and response for each command directed to a Supernova.


### 1. Startup

#### 1.1 Import the python package

In [ ]:
import binhosupernova
from binhosupernova.supernova import Supernova
from binhosupernova.commands.i3c.definitions import *

import random
import threading

print_lock = threading.Lock()

#### 1.2 List all the Supernova devices connected to the PC host

The ``binhosupernova.getConnectedSupernovaDevicesList()`` returns a list of the Supernova devices plugged into the host PC machine.

Two different Supernovas with different serial numbers and paths must appear in the response. Their paths are get and saved in global variables to be used later in the code to open connection.

In [ ]:
connected_devices = binhosupernova.getConnectedSupernovaDevicesList()
print(connected_devices)

if len(connected_devices) < 2:
    print("Two Supernovas connected to this PC are required")

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#6&39aa8cf6&2&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}', 'vendor_id': '0x1fc9', 'product_id': '0x82fc', 'serial_number': 'D6175D813F06E05EA0A6A497E6C926A', 'release_number': 256, 'manufacturer_string': 'Binho LLC', 'product_string': 'Binho Supernova', 'usage_page': 65280, 'usage': 1, 'interface_number': 0}, {'path': '\\\\?\\HID#VID_1FC9&PID_82FC#6&11c48e71&2&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}', 'vendor_id': '0x1fc9', 'product_id': '0x82fc', 'serial_number': '39EB517A3B54BE5E95DBDDCA22B6A16', 'release_number': 256, 'manufacturer_string': 'Binho LLC', 'product_string': 'Binho Supernova', 'usage_page': 65280, 'usage': 1, 'interface_number': 0}]


In [3]:
# Get the USB path of both devices.
I3C_TARGET_USB_PATH = connected_devices[0]["path"]
I3C_CONTROLLER_USB_PATH = connected_devices[1]["path"]

#### 1.3 Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``. The id value `0` is reserved for asynchronous notifications.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [4]:
# Auxiliar code to generate IDs

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 1.4 Define and register callbacks to handle responses and notifications from the two Supernovas (the one acting as an I3C target and the one acting as an I3C controller)

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [5]:
shared_controller_response = {}

# Define callback function
def controller_callback_function(supernova_message: dict, system_message: dict) -> None:

    global shared_controller_response

    with print_lock:

        if supernova_message != None:

            # Print a header
            print(">> New message from Supernova I3C Controller:")
            shared_controller_response = supernova_message
            print(supernova_message)

        if system_message != None:

            # Print a header
            print(">> New message from the SYSTEM:")
            print(system_message)

In [6]:
shared_target_response = {}

# Define callback function
def target_callback_function(supernova_message: dict, system_message: dict) -> None:

    global shared_target_response

    with print_lock:

        if supernova_message != None:

            # Print a header
            print(">> New message from Supernova I3C Target:")

            shared_target_response = supernova_message
            print(supernova_message)

        if system_message != None:

            # Print a header
            print(">> New message from the SYSTEM:")
            print(system_message)

### 2. Configure the Supernova that will act as an I3C target

#### 2.1 Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [7]:
supernova_target = Supernova()

#### 2.2 Open connection to the Supernova that acts as a target using the path of one of the devices retrieved by getConnectedSupernovaDevicesList() 

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``binhosupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [8]:
supernova_target.open(path = I3C_TARGET_USB_PATH)

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 2.3 Register the callback to the I3C target

In [9]:
supernova_target.onEvent(target_callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 2.4 Initialize the Supernova I3C peripheral as a target memory of 1024 registers of 1 byte data width

In [10]:
memory_layout = I3cTargetMemoryLayout_t.MEMORY_1_BYTE_X_1024_REGS

pid = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06]

bcr = I3cBcrDeviceRoleBits_t.I3C_TARGET.value | \
      I3cBcrAdvancedCapabilitiesBit_t.NOT_SUPPORTED.value | \
      I3cBcrVirtualSupportBit_t.NOT_SUPPORTED.value | \
      I3cBcrOfflineCapBit_t.OFFLINE_UNCAPABLE.value | \
      I3cBcrIbiPayloadBit_t.IBI_WITHOUT_PAYLOAD.value | \
      I3cBcrIbiCapableBit_t.NOT_IBI_CAPABLE.value | \
      I3cBcrMaxDataSpeedLimitBit_t.NO_DATA_SPEED_LIMIT.value
print("BCR: ", bcr)

dcr = I3cTargetDcr_t.I3C_TARGET_MEMORY
print("DCR: ", dcr.name)

static_address = 0x21

mwl = 255
mrl = 255

request_result = supernova_target.i3cTargetInit(getId(), memory_layout, pid, bcr, dcr, static_address, mwl, mrl)

BCR:  8
DCR:  I3C_TARGET_MEMORY
>> New message from Supernova I3C Target:
{'id': 1, 'command': 'I3C TARGET INIT', 'result': 'SUCCESS'}


#### 2.5 Take a look at the internal memory

The public method ``Supernova.i3cTargetReadMemory()`` allows the USB Host to read data from the Supernova internal memory.

```python
i3cTargetReadMemory(id, memoryAddress, length)
```

- ``id: c_uint16``: 2-bytes integer from 1 to 65535 that identifies the command.
- ``memoryAddress: c_uint16``: Address of the memory at which data will be read from.
- ``length: c_uint16``: Length of the data to be read. From 1 to 1024 bytes.

In [11]:
supernova_target.i3cTargetReadMemory(getId(),0x0000, 15)

{'module': 1, 'opcode': 0, 'message': 'I3C TARGET READ MEMORY request success'}

>> New message from Supernova I3C Target:
{'id': 2, 'command': 'I3C TARGET READ MEMORY', 'result': 'SUCCESS', 'payload_length': 15, 'payload': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


#### 2.6 Write data into the internal memory

The public method ``Supernova.i3cTargetWriteMemory()`` allows the USB Host to send and write data into the Supernova internal memory.
```python
i3cTargetWriteMemory(id, memoryAddress, data)
```

- ``id: c_uint16``: Identifies the command.
- ``memoryAddress: c_uint16``: Address of the memory at which data will be written from.
- ``data: list``: Data to be written into the Supernova memory.

In [12]:
data = [i for i in range(10,0,-1)]

supernova_target.i3cTargetWriteMemory(getId(), 0x0000, data)

{'module': 1,
 'opcode': 0,
 'message': 'I3C TARGET WRITE MEMORY request success'}

>> New message from Supernova I3C Target:
{'id': 3, 'command': 'I3C TARGET WRITE MEMORY', 'result': 'SUCCESS'}


In [13]:
supernova_target.i3cTargetReadMemory(getId(),0x0000, 15)

{'module': 1, 'opcode': 0, 'message': 'I3C TARGET READ MEMORY request success'}

>> New message from Supernova I3C Target:
{'id': 4, 'command': 'I3C TARGET READ MEMORY', 'result': 'SUCCESS', 'payload_length': 15, 'payload': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0]}


#### 2.7 Data length vs available space

In [14]:
memory_address = random.randint(0,1023)
available_space = 1024 - memory_address

data_length = random.randint(1,1024)
data_to_write = random.choices(range(0,256), k=data_length)

print(f"Random memory address: {memory_address}")
print(f"Available space: {available_space}")
print(f"Create list of random numbers of length {data_length}.")

if available_space >= data_length:
    print("The result will be: SUCCESS\r\n")
else:
    print("The result will be: SURPASSED MEMORY SIZE\r\n")

print("Supernova response:\r\n")

request_result = supernova_target.i3cTargetWriteMemory(getId(), memory_address, data_to_write)

Random memory address: 653
Available space: 371
Create list of random numbers of length 468.
The result will be: SURPASSED MEMORY SIZE

Supernova response:



>> New message from Supernova I3C Target:
{'id': 5, 'command': 'I3C TARGET WRITE MEMORY', 'result': 'SURPASSED_MEMORY_SIZE'}


In [15]:
request_result = supernova_target.i3cTargetReadMemory(getId(), memory_address, data_length)

>> New message from Supernova I3C Target:
{'id': 6, 'command': 'I3C TARGET READ MEMORY', 'result': 'SURPASSED_MEMORY_SIZE', 'payload_length': 371, 'payload': [158, 78, 211, 217, 96, 246, 14, 212, 139, 164, 114, 119, 169, 52, 61, 213, 159, 39, 226, 87, 76, 142, 98, 204, 71, 109, 80, 94, 237, 50, 167, 26, 44, 70, 149, 34, 36, 68, 18, 42, 167, 173, 95, 89, 175, 87, 67, 247, 14, 92, 247, 210, 66, 1, 155, 8, 133, 56, 17, 242, 165, 85, 123, 127, 182, 241, 189, 203, 181, 63, 238, 130, 85, 202, 42, 181, 204, 45, 222, 189, 168, 151, 156, 129, 125, 189, 56, 6, 185, 49, 198, 89, 136, 136, 114, 228, 42, 113, 71, 187, 88, 140, 73, 156, 0, 8, 152, 93, 32, 246, 87, 135, 134, 18, 63, 40, 130, 177, 178, 210, 201, 153, 210, 127, 58, 14, 40, 108, 209, 168, 78, 5, 171, 223, 251, 121, 197, 226, 68, 212, 178, 202, 76, 215, 18, 233, 19, 156, 42, 164, 85, 39, 144, 241, 253, 233, 24, 50, 149, 196, 2, 228, 172, 157, 194, 228, 158, 16, 235, 53, 125, 200, 167, 143, 93, 250, 201, 97, 128, 27, 112, 128, 55, 22, 84,

In [16]:
if shared_target_response["payload"] == data_to_write[:shared_target_response["payload_length"]]:
    print("Data transferred successfully!")

Data transferred successfully!


### 3. Configure the Supernova that will act as an I3C controller

#### 3.1 Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [17]:
supernova_controller = Supernova()

#### 3.2 Open connection to the Supernova that acts as a controller using the path of one of the devices retrieved by getConnectedSupernovaDevicesList() 

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``binhosupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [18]:
supernova_controller.open(path = I3C_CONTROLLER_USB_PATH)

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 3.3 Register the callback to the I3C controller

In [19]:
supernova_controller.onEvent(controller_callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 3.4 Initialize the Supernova I3C peripheral as controller

``Supernova.i3cControllerInit(id, pushPullRate, i3cOpenDrainRate, i2cOpenDrainRate)``

This method initializes the Supernova as an I3C controller.

  - `id: int`: A 2-byte integer that represents the transfer ID.
  - `pushPullRate: I3cPushPullTransferRate`: Push-Pull frequency.
  - `i3cOpenDrainRate: I3cOpenDrainTransferRate`: I3C frequency in Open Drain mode.
  - `i2cOpenDrainRate: I2cTransferRate`: I2C frequency.

In [20]:
request_result = supernova_controller.i3cControllerInit(getId(), I3cPushPullTransferRate.PUSH_PULL_5_MHZ, I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ, I2cTransferRate._100KHz)

>> New message from Supernova I3C Controller:
{'id': 7, 'command': 'I3C CONTROLLER INIT', 'result': 'SUCCESS'}


#### 3.5 Set I3C bus voltage

By default, both Supernovas work at 1.62V. If it is necessary to change the bus voltage, the voltage must be set in both devices.

In [21]:
#Uncomment lines below if it is required to change the I3C bus voltage.

# Set the voltage of the Supernova acting as a controller.
supernova_controller.setI3cVoltage(getId(), 3300)

{'module': 1, 'opcode': 0, 'message': 'SET I3C VOLTAGE request success'}

>> New message from Supernova I3C Controller:
{'id': 8, 'command': 'SYS SET I3C VOLTAGE', 'result': 'SUCCESS'}


In [22]:
# In the case of the target, use one of the following methods.
# supernova_target.setI3cVoltage(getId(), 3300)
supernova_target.useExternalI3cVoltage(getId())

{'module': 1,
 'opcode': 0,
 'message': 'USE EXT SRC I3C VOLTAGE request success'}

>> New message from Supernova I3C Target:
{'id': 9, 'command': 'SYS USE EXTERNAL I3C VOLTAGE', 'result': 'SUCCESS', 'i3c_low_voltage_mV': 1, 'i3c_high_voltage_mV': 3182}


#### 3.6 I3C bus initialization

In [23]:
request_result = supernova_controller.i3cControllerResetBus(getId())

>> New message from Supernova I3C Controller:
{'id': 10, 'command': 'I3C CONTROLLER RESET BUS', 'result': 'SUCCESS'}


In [24]:
dynamic_address = 0x73

request_result = supernova_controller.i3cSETDASA(getId(), static_address, dynamic_address)

>> New message from Supernova I3C Controller:
{'id': 11, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETDASA', 'payload_length': 1}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 0}


>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_TARGET_DYN_ADDR_CHANGE_EVENT', 'new_address': 115}


In [25]:
request_result = supernova_controller.i3cControllerGetTargetDevicesTable(getId())

>> New message from Supernova I3C Controller:
{'id': 12, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 1, 'table': [{'static_address': 33, 'dynamic_address': 115, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'REJECT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}]}


#### 3.7 I3C CCC Transfers

##### 3.7.1 GETPID

In [26]:
request_result = supernova_controller.i3cGETPID(getId(), dynamic_address)

>> New message from Supernova I3C Controller:
{'id': 13, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETPID', 'payload_length': 6, 'payload': [1, 2, 3, 4, 5, 6]}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


##### 3.7.2 GETBCR

In [27]:
request_result = supernova_controller.i3cGETBCR(getId(), dynamic_address)

>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}
>> New message from Supernova I3C Controller:
{'id': 14, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETBCR', 'payload_length': 1, 'payload': [8]}


##### 3.7.3 GETDCR

In [28]:
request_result = supernova_controller.i3cGETDCR(getId(), dynamic_address)

>> New message from Supernova I3C Controller:


{'id': 15, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETDCR', 'payload_length': 1, 'payload': [197]}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


In [29]:
# Check DCR value meaning.
I3cTargetDcr_t(shared_controller_response["payload"][0]).name

'I3C_TARGET_MEMORY'

##### 3.7.4 GETMWL

In [30]:
request_result = supernova_controller.i3cGETMWL(getId(), dynamic_address)

>> New message from Supernova I3C Controller:
{'id': 16, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMWL', 'payload_length': 2, 'payload': [0, 255]}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


##### 3.7.5 SETMWL

In [31]:
NEW_MWL         = 1024

request_result = supernova_controller.i3cDirectSETMWL(getId(), dynamic_address, NEW_MWL)

>> New message from Supernova I3C Controller:
{'id': 17, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETMWL', 'payload_length': 2}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


##### 3.7.6 GETMWL

In [32]:
request_result = supernova_controller.i3cGETMWL(getId(), dynamic_address)

>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}
>> New message from Supernova I3C Controller:
{'id': 18, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMWL', 'payload_length': 2, 'payload': [4, 0]}


##### 3.7.7 GETMRL

In [33]:
request_result = supernova_controller.i3cGETMRL(getId(), dynamic_address)

>> New message from Supernova I3C Controller:
{'id': 19, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMRL', 'payload_length': 2, 'payload': [0, 255]}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


##### 3.7.8 SETMRL

In [34]:
NEW_MRL         = 512

request_result = supernova_controller.i3cDirectSETMRL(getId(), dynamic_address, NEW_MRL)

>> New message from Supernova I3C Controller:
{'id': 20, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETMRL', 'payload_length': 2}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


##### 3.7.9 GETMRL

In [35]:
request_result = supernova_controller.i3cGETMRL(getId(), dynamic_address)

>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


>> New message from Supernova I3C Controller:
{'id': 21, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMRL', 'payload_length': 2, 'payload': [2, 0]}


##### 3.7.10 GETSTATUS

In [36]:
request_result = supernova_controller.i3cGETSTATUS(getId(), dynamic_address)

>> New message from Supernova I3C Controller:
{'id': 22, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETSTATUS', 'payload_length': 2, 'payload': [0, 0]}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


#### 3.8 Set I3C Target parameters

The public method ``Supernova.i3cTargetSetParameters()`` sets the configuration of the supernova acting as a target on the I3C bus.

In [37]:
memory_layout = I3cTargetMemoryLayout_t.MEMORY_1_BYTE_X_1024_REGS

pid = [0xF1, 0xF2, 0xF3, 0xF4, 0xF5, 0xF6]

bcr = I3cBcrDeviceRoleBits_t.I3C_TARGET.value | \
      I3cBcrAdvancedCapabilitiesBit_t.NOT_SUPPORTED.value | \
      I3cBcrVirtualSupportBit_t.NOT_SUPPORTED.value | \
      I3cBcrOfflineCapBit_t.OFFLINE_UNCAPABLE.value | \
      I3cBcrIbiPayloadBit_t.IBI_WITHOUT_PAYLOAD.value | \
      I3cBcrIbiCapableBit_t.NOT_IBI_CAPABLE.value | \
      I3cBcrMaxDataSpeedLimitBit_t.NO_DATA_SPEED_LIMIT.value
print("BCR: ", bcr)

dcr = I3cTargetDcr_t.I3C_TARGET_MEMORY
print("DCR: ", dcr.name)

static_address = 0x21

mwl = 32
mrl = 64

request_result = supernova_target.i3cTargetSetParameters(getId(), memory_layout, pid, bcr, dcr, static_address, mwl, mrl)

BCR:  8
DCR:  I3C_TARGET_MEMORY
>> New message from Supernova I3C Target:
{'id': 23, 'command': 'I3C TARGET SET PARAMETERS', 'result': 'SUCCESS'}


3.8.1 Check the correct setting of the MWL using the method `i3cTargetSetConfiguration` above by requesting a GETMWL CCC.

In [38]:
request_result = supernova_controller.i3cGETMWL(getId(), dynamic_address)

>> New message from Supernova I3C Controller:
{'id': 24, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMWL', 'payload_length': 2, 'payload': [0, 32]}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


3.8.2 Check the correct setting of the MRL using the method `i3cTargetSetConfiguration` above by requesting a GETMRL CCC.

In [39]:
request_result = supernova_controller.i3cGETMRL(getId(), dynamic_address)

>> New message from Supernova I3C Controller:
{'id': 25, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMRL', 'payload_length': 2, 'payload': [0, 64]}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


3.8.3 Check the correct setting of the new PID using the method `i3cTargetSetConfiguration` above by requesting a GETPID CCC.

In [40]:
request_result = supernova_controller.i3cGETPID(getId(), dynamic_address)

>> New message from Supernova I3C Controller:
{'id': 26, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETPID', 'payload_length': 6, 'payload': [241, 242, 243, 244, 245, 246]}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CCC_TRANSFER_EVENT', 'target_address': 115}


#### 3.9 I3C Private Transfers

3.9.1 Previously, let's write random data into the internal memory of the target from the USB Host

In [42]:
memory_address = random.randint(0,1023)
available_space = 1024 - memory_address

data_length = random.randint(1,1024)
data_to_write = random.choices(range(0,256), k=data_length)

print(f"Random memory address: {memory_address}")
print(f"Available space: {available_space}")
print(f"Create list of random numbers of length {data_length}.")

if available_space >= data_length:
    print("The result will be: SUCCESS\r\n")
else:
    print("The result will be: SURPASSED MEMORY SIZE\r\n")

print("Supernova response:\r\n")


request_result = supernova_target.i3cTargetWriteMemory(getId(), memory_address, data_to_write)

Random memory address: 133
Available space: 891
Create list of random numbers of length 224.
The result will be: SUCCESS

Supernova response:



>> New message from Supernova I3C Target:
{'id': 28, 'command': 'I3C TARGET WRITE MEMORY', 'result': 'SUCCESS'}


3.9.2 Now, let's read the written random data to compare with the data read by the I3C Controller.

In [43]:
request_result = supernova_target.i3cTargetReadMemory(getId(), memory_address, data_length)

>> New message from Supernova I3C Target:
{'id': 29, 'command': 'I3C TARGET READ MEMORY', 'result': 'SUCCESS', 'payload_length': 224, 'payload': [129, 181, 184, 234, 152, 229, 95, 123, 172, 166, 19, 245, 176, 71, 252, 92, 225, 71, 42, 99, 255, 212, 72, 224, 10, 244, 152, 50, 79, 142, 162, 252, 15, 149, 79, 206, 12, 72, 18, 128, 204, 242, 192, 77, 10, 36, 184, 73, 31, 94, 114, 23, 200, 130, 17, 177, 163, 125, 35, 171, 115, 229, 14, 29, 195, 14, 209, 121, 88, 90, 52, 23, 153, 231, 55, 74, 146, 61, 106, 86, 140, 30, 141, 164, 71, 129, 32, 144, 198, 96, 39, 41, 197, 44, 156, 12, 8, 218, 120, 204, 121, 39, 206, 104, 107, 226, 26, 5, 52, 230, 45, 44, 140, 39, 252, 27, 172, 81, 214, 172, 146, 186, 215, 235, 55, 91, 101, 28, 81, 199, 241, 146, 154, 133, 201, 203, 36, 70, 106, 95, 213, 143, 136, 76, 64, 86, 0, 148, 219, 45, 149, 74, 6, 33, 231, 87, 108, 212, 195, 117, 43, 39, 54, 63, 76, 70, 92, 168, 149, 215, 157, 20, 60, 75, 31, 83, 16, 15, 85, 198, 145, 213, 223, 141, 236, 14, 139, 239, 245,

In [44]:
data_read_by_host = shared_target_response["payload"][:shared_target_response["payload_length"]]
print(data_read_by_host)

[129, 181, 184, 234, 152, 229, 95, 123, 172, 166, 19, 245, 176, 71, 252, 92, 225, 71, 42, 99, 255, 212, 72, 224, 10, 244, 152, 50, 79, 142, 162, 252, 15, 149, 79, 206, 12, 72, 18, 128, 204, 242, 192, 77, 10, 36, 184, 73, 31, 94, 114, 23, 200, 130, 17, 177, 163, 125, 35, 171, 115, 229, 14, 29, 195, 14, 209, 121, 88, 90, 52, 23, 153, 231, 55, 74, 146, 61, 106, 86, 140, 30, 141, 164, 71, 129, 32, 144, 198, 96, 39, 41, 197, 44, 156, 12, 8, 218, 120, 204, 121, 39, 206, 104, 107, 226, 26, 5, 52, 230, 45, 44, 140, 39, 252, 27, 172, 81, 214, 172, 146, 186, 215, 235, 55, 91, 101, 28, 81, 199, 241, 146, 154, 133, 201, 203, 36, 70, 106, 95, 213, 143, 136, 76, 64, 86, 0, 148, 219, 45, 149, 74, 6, 33, 231, 87, 108, 212, 195, 117, 43, 39, 54, 63, 76, 70, 92, 168, 149, 215, 157, 20, 60, 75, 31, 83, 16, 15, 85, 198, 145, 213, 223, 141, 236, 14, 139, 239, 245, 241, 95, 244, 126, 175, 136, 111, 12, 129, 16, 179, 42, 28, 107, 92, 167, 82, 207, 84, 23, 239, 121, 247, 102, 97, 244, 194, 247, 166, 100, 84, 

3.9.3 If required, modify the MWL and MRL from the USB Host or from the I3C Controller (SETMRL, SETMWL CCCs). In this case we are setting them from the USB Host.

In [45]:
memory_layout = I3cTargetMemoryLayout_t.MEMORY_1_BYTE_X_1024_REGS

pid = [0xF1, 0xF2, 0xF3, 0xF4, 0xF5, 0xF6]

bcr = I3cBcrDeviceRoleBits_t.I3C_TARGET.value | \
      I3cBcrAdvancedCapabilitiesBit_t.NOT_SUPPORTED.value | \
      I3cBcrVirtualSupportBit_t.NOT_SUPPORTED.value | \
      I3cBcrOfflineCapBit_t.OFFLINE_UNCAPABLE.value | \
      I3cBcrIbiPayloadBit_t.IBI_WITHOUT_PAYLOAD.value | \
      I3cBcrIbiCapableBit_t.NOT_IBI_CAPABLE.value | \
      I3cBcrMaxDataSpeedLimitBit_t.NO_DATA_SPEED_LIMIT.value
print("BCR: ", bcr)

dcr = I3cTargetDcr_t.I3C_TARGET_MEMORY
print("DCR: ", dcr.name)

static_address = 0x21

mwl = 1024
mrl = 1024

request_result = supernova_target.i3cTargetSetParameters(getId(), memory_layout, pid, bcr, dcr, static_address, mwl, mrl)

BCR:  8
DCR:  I3C_TARGET_MEMORY
>> New message from Supernova I3C Target:
{'id': 30, 'command': 'I3C TARGET SET PARAMETERS', 'result': 'SUCCESS'}


##### 3.9.4 I3C Private Write with empty data

Update the internal pointer to the internal register address provided in the method below.

In [46]:
mode = TransferMode.I3C_SDR
register_address = [ (memory_address & 0x00FF), ((memory_address & 0xFF00) >> 8) ] # LSB first.
data = []

request_result = supernova_controller.i3cControllerWrite(getId(),dynamic_address,mode,register_address, data)

>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_WRITE_TRANSFER_EVENT', 'target_address': 115, 'memory_address': 133, 'payload_length': 0, 'payload': []}
>> New message from Supernova I3C Controller:
{'id': 31, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 0}


##### 3.9.5 I3C Private Read with empty register address.

The target will assume that the start of the new operation is the end of the previous one.

In [47]:
request_result = supernova_controller.i3cControllerRead(getId(), dynamic_address, mode, [], data_length)

>> New message from Supernova I3C Controller:
{'id': 32, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 224, 'payload': [129, 181, 184, 234, 152, 229, 95, 123, 172, 166, 19, 245, 176, 71, 252, 92, 225, 71, 42, 99, 255, 212, 72, 224, 10, 244, 152, 50, 79, 142, 162, 252, 15, 149, 79, 206, 12, 72, 18, 128, 204, 242, 192, 77, 10, 36, 184, 73, 31, 94, 114, 23, 200, 130, 17, 177, 163, 125, 35, 171, 115, 229, 14, 29, 195, 14, 209, 121, 88, 90, 52, 23, 153, 231, 55, 74, 146, 61, 106, 86, 140, 30, 141, 164, 71, 129, 32, 144, 198, 96, 39, 41, 197, 44, 156, 12, 8, 218, 120, 204, 121, 39, 206, 104, 107, 226, 26, 5, 52, 230, 45, 44, 140, 39, 252, 27, 172, 81, 214, 172, 146, 186, 215, 235, 55, 91, 101, 28, 81, 199, 241, 146, 154, 133, 201, 203, 36, 70, 106, 95, 213, 143, 136, 76, 64, 86, 0, 148, 219, 45, 149, 74, 6, 33, 231, 87, 108, 212, 195, 117, 43, 39, 54, 63, 76, 70, 92, 168, 149, 215, 157, 20, 60, 75, 31, 83, 16, 15, 85, 198, 145, 213, 223, 141, 236, 14, 1

3.9.6 Check if the data read by the I3C Controller is equal to the data written and read by the USB Host.

In [48]:
data_read_by_controller = shared_controller_response["payload"]

if data_read_by_host == data_to_write and  data_read_by_controller == data_to_write:
    print("Data successfully written by the USB Host and read by the I3C Controller!")

Data successfully written by the USB Host and read by the I3C Controller!


##### 3.9.2 I3C Private Read

In [49]:
mode = TransferMode.I3C_SDR
register_address = [ (memory_address & 0x00FF), ((memory_address & 0xFF00) >> 8) ] # LSB first.

if available_space >= data_length:
    length = data_length
else:
    length = available_space

print(f"Register address: {register_address}")
print(f"Read length: {length}\r\n")

if data_length >= mrl:
    print("The result of the I3C TARGET BUS EVENT NOTIFICATION will be I3C TRANSFER MRL REACHED\r\n")
else:
    print("The result of the I3C TARGET BUS EVENT NOTIFICATION will be I3C TRANSFER ABORTED BY CONTROLLER\r\n")

request_result = supernova_controller.i3cControllerRead(getId(), dynamic_address, mode, register_address, length)

Register address: [133, 0]
Read length: 224

The result of the I3C TARGET BUS EVENT NOTIFICATION will be I3C TRANSFER ABORTED BY CONTROLLER

>> New message from Supernova I3C Controller:
{'id': 33, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'I3C_NACK_ADDRESS', 'payload_length': 0, 'payload': []}
>> New message from Supernova I3C Target:
{'id': 0, 'command': 'I3C TARGET BUS EVENT NOTIFICATION', 'result': 'TX_FIFO_EMPTY', 'event': 'I3C_READ_TRANSFER_EVENT', 'target_address': 115, 'memory_address': 133, 'payload_length': 0, 'payload': []}


##### 3.9.5 I3C Private Write with data

In [50]:
register_address = [0x00, 0x00]
data_to_write = [i for i in range(12)]

request_result = supernova_target.i3cControllerWrite(getId(),dynamic_address, TransferMode.I3C_SDR, register_address, data_to_write)

>> New message from Supernova I3C Target:
{'id': 34, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'BUS_TIMEOUT', 'payload_length': 0}


In [51]:
memory_address = random.randint(0,1023)
available_space = 1024 - memory_address

data_length = random.randint(1,1024)
data_to_write = random.choices(range(0,256), k=data_length)

print(f"Random memory address: {memory_address}")
print(f"Available space: {available_space}")
print(f"Create list of random numbers of length {data_length}.")

if available_space >= data_length and data_length < mwl:
    print("The result of the I3C TARGET BUS EVENT NOTIFICATION will be: SUCCESS\r\n")
elif data_length >= mwl:
    print("The result will be: I3C TRANSFER MWL REACHED\r\n")

print("Supernovas' response:\r\n")

register_address = [ ((memory_address & 0xFF00) >> 8), (memory_address & 0x00FF) ]

request_result = supernova_target.i3cControllerWrite(getId(),dynamic_address, TransferMode.I3C_SDR, register_address, data_to_write)

Random memory address: 872
Available space: 152
Create list of random numbers of length 836.
Supernovas' response:

>> New message from Supernova I3C Target:
{'id': 35, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'I3C_NACK_ADDRESS', 'payload_length': 0}
